<a href="https://colab.research.google.com/github/git-ekeh/guidance_algorithm/blob/main/GuidanceAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import datasets
import transformers
from transformers import DataCollatorWithPadding
from transformers import BertTokenizerFast, AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import load_dataset, Dataset
from textrank_keywords import TextRank4Keyword
from dsw_transformer import Transformer
import pandas as pd
import collections
import itertools
import concurrent.futures
import pickle

In [ ]:
xsum_validation = load_dataset("xsum", split="validation")

xsum_validation_size = len(xsum_validation)

xsum_validation_start, xsum_end = 0, xsum_validation_size

sample_xsum = itertools.islice(xsum_validation, xsum_validation_start, xsum_end)

#validation_features = validation.features

def keywordExtraction(x):
    textrank = TextRank4Keyword()
    textrank.analyze(x, candidate_pos =['NOUN', 'PROPN'], window_size = 4, lower = False)
    guidance_kw = textrank.get_keywords()
    return guidance_kw

def create_dict_with_keywords(example, dataset):

    return {
        "document": example['document'],
        "summary": (example['summary']),
        "guidance": keywordExtraction(example['document'])}


def generate_dataset_dicts(data_iterator, data):
    with concurrent.futures.ThreadPoolExecutor(8) as executor:
        results = list(executor.map(create_dict_with_keywords, data_iterator, itertools.repeat(data)))

    dataset_dict = {str(i): result for i, result in enumerate(results)}
    return dataset_dict

xsum_dict = generate_dataset_dicts(sample_xsum, xsum_validation)

# Save the dictionaries to disk
with open("xsum_validation_with_ranking.pkl", "wb") as f:
    pickle.dump(xsum_dict, f)